In [92]:
from tabulate import tabulate
import numpy as np
import itertools

## reads in a transaction data file and returns a list of lists.
## Each element in the output list is a transaction. This is a list of items contained in this transaction
def readTransactionalDatabase(file):
    db = []
    with open(file) as f:
        for line in f.readlines():
            li = line.split(" ")
            row = []
            for v in li:
                if v != "\n":
                    row.append(int(v))
            row.sort()
            db.append(row)        
    return db

## Computes the set of all items that occur in a database
def getItemset(arr):
    iset = set([])
    for row in arr:
        for val in row:
            if ~np.isnan(val):
                iset.add(int(val))
    a =list(iset)
    a.sort()
    return a

## Turns a transactional database into a vertical one
def verticalizeTransactionDatabase(db):
    cols = {}
    for tid in range(len(db)):
        trans = db[tid]
        for item in trans:
            if not item in cols:
                cols[item] = []
            cols[item].append(tid)
    return cols

## Computes the difference list between list li1 and li2
def diff(li1, li2): 
    li_dif = []
    for e in li1:
        if not e in li2:
            li_dif.append(e)
    return li_dif

## Computes all subsets of size s of a set with n elements
def findsubsets(s, n): 
    return list(itertools.combinations(s, n))

## Checks whether an itemset l2 may be prefixed by an itemset l1
def isValidPrefix(l1, l2):
    if isinstance(l2, list):
        d = diff(l2, l1)
        return l1[len(l1) - 1] < d[0] if len(d) > 0 else False
    else:
        return l1[len(l1) - 1] < l2

## Computes the set of all subsets (lists) of li
def powerset(li):
    ps = itertools.chain.from_iterable(itertools.combinations(li, r) for r in range(len(li)+1))
    pl = []
    for e in ps:
        pl.append(list(e))
    return pl

## Helper function, not to be called directly
def hasSortedListItem(li, item):
    for e in li:
        if e > item:
            return False
        if e == item:
            return True
    return False

## Identifies all transactions in the database that contain a single given item
def getTransactionsWithItem(db, item):
    t = []
    for tid in range(len(db)):
        trans = db[tid]
        if hasSortedListItem(trans, item):
            t.append(tid)
    return t

## Identifies all transactions in the database the contain all of the given items
def getTransactions(db, iset):
    t = []
    isetAsSet = set(iset)
    for tid in range(len(db)):
        trans = db[tid]
        if isetAsSet.issubset(set(trans)):
            t.append(tid)
    return t

In [112]:
%%time
h=['0','A','B','C','D','E']
def apriori(db, minsup):
    f=[]
    c=[[]]
    r=[]
    for i in getItemset(db):
        r.append([[i],0])
    c.append(r)
    k=1
    while len(c[k])!=0:
        ComputeSupport(c[k],db)
        ct=[]
        for leaf in c[k]:
            if (leaf[1])>minsup:
                f.append(leaf)
                ct.append(leaf)
                # print([h[x] for x in leaf[0]],leaf[1])
        ct.sort()
        c[k]=ct
        # print('k',k,'\n',c[k])
        c.append(ExtendPrefixTree(c[k]))
        
        k+=1
    f.sort()
    return f

def ComputeSupport(c,db):
    for i in db:
        for a in range(0,len(c)):
            temp = 0
            for b in c[a][0]:
                if b in i :
                    temp+=1
            if temp ==len(c[a][0]):
                c[a][1]+=1
          
def ExtendPrefixTree(c):
    extensions=True
    cn=[]
    for leafXa in c:
        for leafXb in c:
            if isValidPrefix(leafXa[0],leafXb[0]):
                Xab=list(set(leafXa[0]) | set(leafXb[0]))
                Xab.sort()
                if [Xab,0] not in cn and len(Xab)==len(leafXa[0])+1:
                    cn.append([Xab,0])
                extensions=False
        if extensions:
            c.remove(leafXa)
    return  cn

print('apriori\n',apriori(readTransactionalDatabase("exampleset.dat"),2))

apriori
 [[[1], 4], [[1, 2], 4], [[1, 2, 4], 3], [[1, 2, 4, 5], 3], [[1, 2, 5], 4], [[1, 4], 3], [[1, 4, 5], 3], [[1, 5], 4], [[2], 6], [[2, 3], 4], [[2, 3, 5], 3], [[2, 4], 4], [[2, 4, 5], 3], [[2, 5], 5], [[3], 4], [[3, 5], 3], [[4], 4], [[4, 5], 3], [[5], 5]]
Wall time: 999 µs


In [115]:
%%time
r=['0','A','B','C','D','E']
def eclat( p, minsup,f):
   for i in range (0,len(p)):
        f.append([p[i][0],len(p[i][1])])
        f.sort()
        # print([ [r[x] for x in p[i][0]] , [x+1 for x in p[i][1]] ])
        # print(  p[i][0] ,p[i][1])
        Pa=[]
        for j in range (0,len(p)):
            if p[j][0]>p[i][0]:
                Xab=list(set(p[j][0]) | set(p[i][0]))
                XabT=list(set(p[j][1]) & set(p[i][1]))
                if len(XabT)>minsup:
                    Pa.append([Xab,XabT])
        if len(Pa)!=0:
            eclat(Pa,minsup,f)
   return f
p=[]
temp=verticalizeTransactionDatabase(readTransactionalDatabase("exampleset.dat"))
for i in temp:
    if len(temp[i])>2:
        p.append([[i],temp[i]])
p.sort()
print('eclat\n',eclat(p,2,[]))

eclat
 [[[1], 4], [[1, 2], 4], [[1, 2, 4], 3], [[1, 2, 4, 5], 3], [[1, 2, 5], 4], [[1, 4], 3], [[1, 4, 5], 3], [[1, 5], 4], [[2], 6], [[2, 3], 4], [[2, 3, 5], 3], [[2, 4], 4], [[2, 4, 5], 3], [[2, 5], 5], [[3], 4], [[3, 5], 3], [[4], 4], [[4, 5], 3], [[5], 5]]
Wall time: 999 µs


In [117]:
%%time
r=['0','A','B','C','D','E']
def Declat( p, minsup,f):
   for i in range (0,len(p)):
        f.append([p[i][0],p[i][2][0]])
        # print([ [r[x] for x in p[i][0]] ,p[i][2]], [x+1 for x in p[i][1]] )
        # print(p[i][0],p[i][2],p[i][1])
        Pa=[]
        for j in range (0,len(p)):
            if p[j][0]>p[i][0]:
                Xab=list(set(p[i][0]) | set(p[j][0]))
                dXab=diff(p[j][1],p[i][1])
                supXab=p[i][2][0]-len(dXab)
                if supXab>minsup:
                    Pa.append([Xab,dXab,[supXab]])
        if len(Pa)!=0:
            Declat(Pa,minsup,f)
   return f
p=[]
temp=verticalizeTransactionDatabase(readTransactionalDatabase("exampleset.dat"))
for i in temp:
    if len(temp[i])>2:
        p.append([[i],diff(list(range(0,len(temp)+1)),temp[i]),[len(temp[i])]])
p.sort()
print('Declat\n',Declat(p,2,[]))

Declat
 [[[1], 4], [[1, 2], 4], [[1, 2, 4], 3], [[1, 2, 4, 5], 3], [[1, 2, 5], 4], [[1, 4], 3], [[1, 4, 5], 3], [[1, 5], 4], [[2], 6], [[2, 3], 4], [[2, 3, 5], 3], [[2, 4], 4], [[2, 4, 5], 3], [[2, 5], 5], [[3], 4], [[3, 5], 3], [[4], 4], [[4, 5], 3], [[5], 5]]
Wall time: 1 ms


In [91]:
%%time
r=['0','A','B','C','D','E']
def createStrongRules(fsets, minconf):
    reglas=[]
    for Z in fsets:
        if len(Z[0])>=2:
            A=[]
            for i in range (len(Z[0])-1,0,-1):
                n=findsubsets(Z[0],i)
                for j in range(0,len(n),1):
                    b=list(n[j])
                    for k in fsets:
                        if b==k[0]:
                            A.append(b)
            while len(A)!=0:
                X=A[0]
                A=diff(A,[X])
                temp=0
                for j in fsets:
                    if X==j[0]:
                        temp=j[1]
                c=Z[1]/temp
                if c>minconf :
                    reglas.append([X,diff(Z[0],X),Z[1],c])
                    print('',[r[x] for x in X] ,'----->',[r[x] for x in diff(Z[0],X)] ,Z[1],c)
                else:
                    h=[]
                    for i in range (1,len(X)):
                        for j in findsubsets(X,i):
                            h.append(list(j))
                    A=diff(A,h)
    reglas.sort()
    return reglas
    
def createAssociationRules(fsets, minsup):
    reglas=[]
    for Z in fsets:
        if len(Z[0])>=minsup:
            A=[]
            for i in range (len(Z[0])-1,0,-1):
                n=findsubsets(Z[0],i)
                for j in range(0,len(n),1):
                    b=list(n[j])
                    for k in fsets:
                        if b==k[0]:
                            A.append(b)
            
            while len(A)!=0:
                X=A[0]
                A=diff(A,[X])
                temp=0
                for j in fsets:
                    if X==j[0]:
                        temp=j[1]
                c=Z[1]/temp
                reglas.append([X,diff(Z[0],X),Z[1],c])
    reglas.sort()
    return reglas

def getStrongRules(db,minsup,minconf):
    p=[]
    temp=verticalizeTransactionDatabase(readTransactionalDatabase(db))
    for i in temp:
        if len(temp[i])>=minsup:
            p.append([[i],temp[i]])
    p.sort()
    reglasF=[]
    reglas=createAssociationRules(eclat(p,minsup,[]),minconf)
    for i in reglas:
        if i[3]>minconf:
            reglasF.append(i)
    return reglasF
                
# p=[]
# temp=verticalizeTransactionDatabase(readTransactionalDatabase("exampleset.dat"))
# for i in temp:
#     if len(temp[i])>2:
#         p.append([[i],temp[i]])
# p.sort()
# print(createStrongRules(eclat(p,2,[]),0.9))
# print(getStrongRules("shop.dat",500,0.75))
o=getStrongRules("shop.dat",500,0.75)

print('--------strong rules sorted by support-------------------')
o.sort(key = lambda x: x[2])
print(tabulate(o, headers=['X', 'Y','SUP','CONF']))

print('\n\n\n--------sub-list at least two items appear-------------------\n')
o.sort(key = lambda x: len(x[1]),reverse=True)
print(tabulate(list(filter(lambda x: len(x[1]) >= 2, o)), headers=['X', 'Y','SUP','CONF']))


--------strong rules sorted by support-------------------
X                     Y                       SUP      CONF
--------------------  --------------------  -----  --------
[368, 296]            [829]                   501  0.790221
[494, 815]            [944, 862]              502  0.8032
[862, 815]            [944, 494]              502  0.81626
[944, 494]            [862, 815]              502  0.767584
[944, 494, 815]       [862]                   502  0.907776
[944, 815]            [862, 494]              502  0.801917
[944, 862]            [494, 815]              502  0.76176
[944, 862, 494]       [815]                   502  0.912727
[944, 862, 815]       [494]                   502  0.90942
[411, 350]            [579, 842, 572]         503  0.80609
[411, 572]            [579, 842, 350]         503  0.82459
[411, 572, 350]       [579, 842]              503  0.890265
[572, 350]            [579, 803, 842]         503  0.820555
[572, 350]            [579, 842, 411]         503